In [29]:
import pandas as pd

### Загрузка данных

In [30]:
path = 'data/raw/participants/'

df = pd.read_csv(path+'train_candles.csv')

### Обработка данных

In [ ]:
df.head()

In [31]:
df['ticker'].unique()

array(['AFLT', 'ALRS', 'CHMF', 'GAZP', 'GMKN', 'LKOH', 'MAGN', 'MGNT',
       'MOEX', 'MTSS', 'NVTK', 'PHOR', 'PLZL', 'ROSN', 'RUAL', 'SBER',
       'SIBN', 'T', 'VTBR'], dtype=object)

### Добавляем поле tickers

In [ ]:
import pandas as pd

keywords = {
    'AFLT': ['Аэрофлот', 'Aeroflot', 'AFLT'],
    'ALRS': ['Алроса', 'Alrosa', 'ALRS'],
    'CHMF': ['Северсталь', 'Severstal', 'CHMF'],
    'GAZP': ['Газпром', 'Gazprom', 'GAZP'],
    'GMKN': ['Норильский никель', 'ГМК Норильский никель', 'Norilsk Nickel', 'GMKN'],
    'LKOH': ['Лукойл', 'Lukoil', 'LKOH'],
    'MAGN': ['ММК', 'Магнитогорский металлургический комбинат', 'Magnitogorsk', 'MAGN'],
    'MGNT': ['Магнит', 'Magnit', 'MGNT'],
    'MOEX': ['Московская биржа', 'MOEX'],
    'MTSS': ['МТС', 'MTS', 'MTSS'],
    'NVTK': ['Новатэк', 'Novatek', 'NVTK'],
    'PHOR': ['ФосАгро', 'PhosAgro', 'PHOR'],
    'PLZL': ['Полюс', 'Полюс Золото', 'Polyus', 'PLZL'],
    'ROSN': ['Роснефть', 'Rosneft', 'ROSN'],
    'RUAL': ['Русал', 'Rusal', 'RUAL'],
    'SBER': ['Сбербанк', 'Sberbank', 'SBER'],
    'SIBN': ['Сибур', 'Sibur', 'SIBN'],
    'T': ['Татнефть', 'Tatneft', 'T'],
    'VTBR': ['ВТБ', 'VTB', 'VTBR']
}

path = 'data/raw/participants/'

df_news = pd.read_csv(path+'train_news.csv')

def find_related_tickers(text, keywords):
    """Ищет все активы, которые упоминаются в тексте"""
    related = []
    text_lower = text.lower()
    for ticker, kw_list in keywords.items():
        for kw in kw_list:
            if kw.lower() in text_lower:
                related.append(ticker)
                break
    return ','.join(related)

# Добавляем новую колонку с найденными активами
df_news['tickers'] = df_news['publication'].apply(lambda x: find_related_tickers(x, keywords))

# Проверка
print(df_news[['publication', 'tickers']].head())
print(sorted(df_news['tickers'].unique(), key=lambda x: len(x)))

empty = df[df_news['tickers'] == '']

total = len(df_news)
empty_count = len(empty)

print(f"Пустых tickers: {empty_count}")
print(f"Заполненных tickers: {total-empty_count}")



                                         publication      tickers
0  Тенденции в отрасли. Ключевые российские нефте...    GAZP,LKOH
1  Тенденции в отрасли. Ключевые российские стале...  CHMF,MAGN,T
2  Председатель правления "Газпрома" Алексей Милл...         GAZP
3  Дата и время ВОСА – 24.01.2020 23:59:59  Дата ...             
4  Дата и время собрания - 27.12.2019 00:00:00  Д...             
['', 'T', 'GAZP', 'MTSS', 'VTBR', 'LKOH', 'MGNT', 'SBER', 'MAGN', 'ALRS', 'MOEX', 'AFLT', 'NVTK', 'ROSN', 'PHOR', 'CHMF', 'PLZL', 'RUAL', 'GMKN', 'SIBN', 'MGNT,T', 'GAZP,T', 'SBER,T', 'LKOH,T', 'PHOR,T', 'T,VTBR', 'ALRS,T', 'CHMF,T', 'AFLT,T', 'ROSN,T', 'RUAL,T', 'MAGN,T', 'MTSS,T', 'NVTK,T', 'PLZL,T', 'MOEX,T', 'GMKN,T', 'SIBN,T', 'GAZP,LKOH', 'SBER,VTBR', 'MAGN,MGNT', 'MGNT,MTSS', 'GAZP,MOEX', 'GAZP,SBER', 'MOEX,SBER', 'MAGN,SBER', 'LKOH,SBER', 'CHMF,MAGN', 'CHMF,GAZP', 'MAGN,MOEX', 'RUAL,VTBR', 'LKOH,PLZL', 'MGNT,PLZL', 'CHMF,PHOR', 'LKOH,VTBR', 'GAZP,MAGN', 'GAZP,NVTK', 'MTSS,VTBR', 'LKOH,MTSS

/var/folders/td/cbfq31q93ln9cyfyjyjb5qvh0000gn/T/ipykernel_35588/1412294609.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  empty = df[df_news['tickers'] == '']
